In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import os
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# noinspection PyTypeChecker
class ConvolutionalNeuralNet(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  # 3: colors - R G B, 6: output layer size, 5: convolution kernel size
        self.pool = nn.MaxPool2d(4, 4)  #4: Pool size, 4: Stride size
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.LazyLinear(120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

        self.model_feature_learning = torch.nn.Sequential(
            self.conv1,
            nn.ReLU(),
            self.pool,
            self.conv2,
            nn.ReLU(),
            self.pool
        )

        self.model_classification = torch.nn.Sequential(
            self.fc1,
            nn.ReLU(),
            self.fc2,
            nn.ReLU(),
            self.fc3
        )

    def forward(self, x):  # n: number samples in a batch.
        # Start with n, 3, 32, 32
        x = self.model_feature_learning(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.model_classification(x)
        return x


writer = SummaryWriter('runs/mnist')
# Use local GPU for CNN models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
# Set ML configuration
image_size = 400
data_dir = rf'../Pneumonia_classification_data/reshape_{image_size}'
num_epochs = 10
batch_size = 4
rate_learning = 0.001
classes = ('Normal', 'Pneumonia')

In [31]:
# Load and transform datasets
# Images are processed from main.py to 300x300 greyscale jpeg format
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')


transform = transforms.Compose(
    [transforms.Grayscale(num_output_channels=1),
     transforms.ToTensor()])
dataset_test = datasets.ImageFolder(test_dir, transform=transform)
dataset_train = datasets.ImageFolder(train_dir, transform=transform)
dataset_val = datasets.ImageFolder(val_dir, transform=transform)

loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

In [32]:
# Display a batch of data
examples = iter(loader_test)
example_data, example_lable = examples.next()
img_grid = torchvision.utils.make_grid(example_data)
writer.add_image('test_images', img_grid)
writer.close()

In [33]:
# Setup model, loss, optimizer and total training steps
model = ConvolutionalNeuralNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=rate_learning)
n_total_steps = len(loader_train)

example_data = example_data.to(device)
writer.add_graph(model, example_data)
writer.close()

In [34]:
# Start model training
running_loss = 0.0
running_correct = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        running_correct += (predicted == labels).sum().item()

        if (i + 1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            writer.add_scalar('Loss/train', running_loss / 100, epoch * n_total_steps + i)
            writer.add_scalar('Accuracy/train', running_correct / 100, epoch * n_total_steps + i)
            running_loss = 0.0
            running_correct = 0
writer.close()
print('Finished Training')

Epoch [1/10], Step [100/1277], Loss: 0.6807
Epoch [1/10], Step [200/1277], Loss: 0.6501
Epoch [1/10], Step [300/1277], Loss: 0.4717
Epoch [1/10], Step [400/1277], Loss: 0.3850
Epoch [1/10], Step [500/1277], Loss: 0.5494
Epoch [1/10], Step [600/1277], Loss: 0.3240
Epoch [1/10], Step [700/1277], Loss: 0.5674
Epoch [1/10], Step [800/1277], Loss: 0.3116
Epoch [1/10], Step [900/1277], Loss: 0.5714
Epoch [1/10], Step [1000/1277], Loss: 0.3128
Epoch [1/10], Step [1100/1277], Loss: 0.7958
Epoch [1/10], Step [1200/1277], Loss: 1.1256
Epoch [2/10], Step [100/1277], Loss: 0.5389
Epoch [2/10], Step [200/1277], Loss: 0.5446
Epoch [2/10], Step [300/1277], Loss: 0.7644
Epoch [2/10], Step [400/1277], Loss: 1.0769
Epoch [2/10], Step [500/1277], Loss: 0.7936
Epoch [2/10], Step [600/1277], Loss: 0.5366
Epoch [2/10], Step [700/1277], Loss: 1.0597
Epoch [2/10], Step [800/1277], Loss: 0.4853
Epoch [2/10], Step [900/1277], Loss: 0.2589
Epoch [2/10], Step [1000/1277], Loss: 0.4855
Epoch [2/10], Step [1100/127

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(2)]
    n_class_samples = [0 for i in range(2)]
    for images, labels in loader_test:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predictions == labels).sum().item()

        for i in range(len(labels)):
            label = labels[i]
            pred = predictions[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(2):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc:.2f} %')